In [1]:
#use LangSmith
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "langhain-ollama-prompts"

In [3]:
prompt = """
Answer the user's query based on the context below.                 
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "{query}"),
])



In [5]:
#the same as previous but using MessagePromptTemplates
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt),
    HumanMessagePromptTemplate.from_template("{query}"),
])

In [6]:
prompt_template.input_variables

['context', 'query']

In [7]:
prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the user\'s query based on the context below.                 \nIf you cannot answer the question using the\nprovided information answer with "I don\'t know".\n\nContext: {context}\n'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]

Invoke LLM with templates

In [8]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2"

#ititialize more determenistic model
llm = ChatOllama(model=model_name, temperature=0.0)

In [9]:
pipeline = prompt_template | llm

In [10]:
context = """LangChain is a framework for developing applications powered by language models. It can be used for chatbots, Generative Question-Answering (GQA), summarization, and much more."""
query = "What is LangChain used for?"

In [11]:
pipeline.invoke({"context": context, "query": query})

AIMessage(content='LangChain is a framework that can be used for various natural language processing (NLP) tasks, including:\n\n1. Chatbots\n2. Generative Question-Answering (GQA)\n3. Summarization\n\nThese are just a few examples of the many applications LangChain can be used for.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-12-04T09:18:58.91637Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2705030417, 'load_duration': 1591120500, 'prompt_eval_count': 103, 'prompt_eval_duration': 154594583, 'eval_count': 63, 'eval_duration': 660613037, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'}, id='lc_run--581bc0ff-67b3-4ffc-bd53-e2e1b65fda11-0', usage_metadata={'input_tokens': 103, 'output_tokens': 63, 'total_tokens': 166})

Few Shot Prompting

In [17]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

In [13]:
examples = [
    {
        "input": "What is the capital of France?",
        "output": "The capital of France is Paris."
    },
    {
        "input": "Who wrote 'To Kill a Mockingbird'?",
        "output": "'To Kill a Mockingbird' was written by Harper Lee."
    }
]

In [18]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

print(few_shot_prompt.format())

Human: What is the capital of France?
AI: The capital of France is Paris.
Human: Who wrote 'To Kill a Mockingbird'?
AI: 'To Kill a Mockingbird' was written by Harper Lee.


In [19]:
new_system_prompt = """
Answer the user's query based on the context below.                 
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt_template.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"context": context, "query": query}).content
print(out)

**Overview of LangChain**

LangChain is a framework that enables the development of applications powered by language models. It provides a set of tools and libraries to build various types of applications.

### Key Use Cases

*   **Chatbots**: LangChain can be used to build conversational interfaces, enabling users to interact with applications through natural language.
*   **Generative Question-Answering (GQA)**: The framework allows developers to create systems that generate questions and answers based on a given prompt or context.
*   **Summarization**: LangChain can be utilized for text summarization tasks, such as condensing long documents into concise summaries.

### Other Applications

*   **Content Generation**: LangChain can be used to generate content, such as articles, social media posts, or product descriptions.
*   **Language Translation**: The framework provides tools for building language translation systems that can translate text from one language to another.
*   **Tex

In [20]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [21]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

In [22]:
from IPython.display import Markdown, display

out = few_shot_prompt.format()
display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [23]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

In [24]:
pipeline = prompt_template | llm
out = pipeline.invoke({"context": context, "query": query}).content
display(Markdown(out))

## LangChain

LangChain is a framework for developing applications powered by language models.

### Applications

* **Chatbots**: LangChain can be used to build conversational interfaces that use natural language processing (NLP) and machine learning.
* **Generative Question-Answering (GQA)**: LangChain enables the creation of AI systems that can answer questions based on their understanding of a given text or context.
* **Summarization**: LangChain can be used to summarize long pieces of text into shorter, more digestible versions.

### Key Features

* **Modular Architecture**: LangChain's modular design allows developers to easily integrate different components and tools into their applications.
* **Language Model Integration**: LangChain provides a simple way to integrate language models into applications, making it easy to build AI-powered interfaces.
* **Extensive Library of Tools**: LangChain comes with an extensive library of tools and pre-built components that can be used to speed up development.

**To conclude**, LangChain is a powerful framework for building AI-powered applications that use language models.

Chain of Thought Prompting

In [25]:
#test LLM without Chain of Thought prompting
simple_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

simple_prompt_template = ChatPromptTemplate.from_messages([
    ("system", simple_prompt),
    ("user", "{query}"),
])

In [26]:
query = (
    "James has 7 apples, he eats 4 and is given an additional 19 apples, "
    "James gives 15 apples to Josh, and Josh gives James 2 apples, how "
    "many apples does James have?" 
)

simple_pipeline = simple_prompt_template | llm
simple_out = simple_pipeline.invoke({"query": query}).content
print(simple_out)

9


In [27]:
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.

Context: {context}
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | llm
cot_out = cot_pipeline.invoke({"context": context, "query": query}).content
display(Markdown(cot_out))

To solve this problem, we need to break it down into smaller subproblems. Here are the steps:

**Subproblem 1: Initial number of apples**
James starts with 7 apples.

**Subproblem 2: Apples eaten by James**
James eats 4 apples, so he now has:
7 - 4 = 3 apples

**Subproblem 3: Additional apples given to James**
James is given an additional 19 apples, so he now has:
3 + 19 = 22 apples

**Subproblem 4: Apples given by Josh to James**
Josh gives James 2 apples, so James now has:
22 + 2 = 24 apples

**Subproblem 5: Apples given by James to Josh**
James gives 15 apples to Josh, so he now has:
24 - 15 = 9 apples

Therefore, after all the transactions, James has **9 apples**.